In [22]:
sum = 0
for i in range(40):
    print(i)
    sum += 60 / (1.05)**(i+1)
sum

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


1029.5451812396657

In [23]:
sum+=1000 / (1.05)**(40)
sum

1171.5908635399433

In [24]:
sum = 0
for i in range(60):
    sum += 40 / 1.05**(i+1)
sum

757.171581002804

In [27]:
1000 / (1.05)**(60)

53.53552374649414

In [28]:
150 / 1.1

136.36363636363635

In [29]:
3.5*21

73.5

In [34]:
1000 *(1-0.0495/2)

975.25

In [1]:
def Cp(p, rss):
    sigma = rss / (66 - p - 1)
    cp = (rss + 2 * p * sigma) / 66
    return cp


print(Cp(3, 12))
print(Cp(4, 10.2))
Cp(5, 10)

0.19941348973607037
0.17481371087928463


0.17676767676767677

In [2]:
print(1 - 12 / 50)
print(1 - 10.2 / 50)
print(1 - 10 / 50)

0.76
0.796
0.8


In [3]:
import numpy as np
np.exp(-0.5) / (1 + np.exp(-0.5))

0.37754066879814546

In [4]:
(4 * np.exp(-0.5) / (np.exp(-2 / 9) + 4 * np.exp(-0.5)))

0.7518524532975261

In [12]:
import torch
import numpy as np
from tqdm import tqdm
from phe import paillier
tensor = torch.Tensor([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
np.array(tensor).shape

(3, 4)

In [13]:
model = torch.nn.Sequential(
    torch.nn.Linear(20, 10),
    torch.nn.ReLU(),
)
dict1 = model.state_dict()
dict1

OrderedDict([('0.weight',
              tensor([[ 0.1152, -0.1494,  0.0746,  0.1382,  0.0954, -0.2129, -0.0364, -0.0303,
                       -0.0134,  0.0903, -0.1527,  0.1547,  0.0439, -0.1754, -0.0059, -0.1625,
                        0.1836, -0.1389, -0.1688,  0.0078],
                      [-0.1970,  0.2144,  0.1270,  0.1825,  0.0266,  0.1565, -0.2115, -0.2190,
                        0.1667,  0.1963,  0.2194, -0.0482, -0.1382,  0.1716,  0.0684, -0.1222,
                       -0.0772,  0.0579, -0.1700, -0.1036],
                      [ 0.1800, -0.1441, -0.1091,  0.0511,  0.0451,  0.0439, -0.0759,  0.1657,
                        0.1361, -0.0696,  0.1651, -0.1858, -0.1468,  0.1681,  0.0479,  0.1335,
                        0.0235,  0.2120, -0.0235,  0.1609],
                      [ 0.0635,  0.1810,  0.0416,  0.0279,  0.0858, -0.1147, -0.1616, -0.0342,
                       -0.0314,  0.0787, -0.1648,  0.1374, -0.2007,  0.1755,  0.1366,  0.2135,
                       -0.0906,  0

In [29]:
def encrypt_random_vector(vector):
    """
    Encrypts an array with random shape
    :param vector: a numpy array
    :return: the encrypted array reshaped to 1d, the private keyring and the shape of the array
    """
    vector_shape = vector.shape
    vector_1d = vector.reshape(-1)
    keyring = paillier.PaillierPrivateKeyring()
    public_key = []
    encrypted_list = []
    for _ in range(len(vector_1d)):
        pub, priv = paillier.generate_paillier_keypair()
        public_key.append(pub)
        keyring.add(priv)
    for i in tqdm(range(len(vector_1d))):
        encrypted_list.append(public_key[i].encrypt(vector_1d[i]))
    return encrypted_list, keyring, vector_shape

def decrypt_random_vector(encrypted):
    """
    decrypts an array with random shape using the result of the encryption function
    :param encrypted: the result of the encryption function
    :return: the decrypted array
    """
    encrypted_list, keyring, vector_shape = encrypted[0], encrypted[1], encrypted[2]
    decrypted_list = []
    for i in tqdm(range(len(encrypted_list))):
        decrypted_list.append(keyring.decrypt(encrypted_list[i]))
    decrypted_vector = np.array(decrypted_list).reshape(vector_shape)
    return decrypted_vector

In [34]:
def encrypt_paramter_dict(w_dict):
    """we encrypt the parameter tensors in dictionary"""
    key_list, tuple_list = [], []
    for key, value in tqdm(w_dict.items()):
        print(value)
        print(np.array(value, dtype=np.float64))
        tuple_encryption = encrypt_random_vector(np.array(value, dtype=np.float64))
        tuple_list.append(tuple_encryption)
        key_list.append(key)
    return key_list, tuple_list

def decypt_order_dict(encrypted_pair):
    """Designed to decrypt the dict of the global parameters"""
    # 传入的参数是一对列表，第一个列表中包含字典的key，第二个包含一系列tuple用于解密
    key_list, tuple_list = encrypted_pair[0], encrypted_pair[1]
    decrypted_dict = {}
    for index, en_tuple in enumerate(tuple_list):
        decrypted_dict[key_list[index]] = decrypt_random_vector(en_tuple)
    return decrypted_dict

In [35]:
pair = encrypt_paramter_dict(dict1)
pair

  0%|          | 0/2 [00:00<?, ?it/s]

tensor([[ 0.1152, -0.1494,  0.0746,  0.1382,  0.0954, -0.2129, -0.0364, -0.0303,
         -0.0134,  0.0903, -0.1527,  0.1547,  0.0439, -0.1754, -0.0059, -0.1625,
          0.1836, -0.1389, -0.1688,  0.0078],
        [-0.1970,  0.2144,  0.1270,  0.1825,  0.0266,  0.1565, -0.2115, -0.2190,
          0.1667,  0.1963,  0.2194, -0.0482, -0.1382,  0.1716,  0.0684, -0.1222,
         -0.0772,  0.0579, -0.1700, -0.1036],
        [ 0.1800, -0.1441, -0.1091,  0.0511,  0.0451,  0.0439, -0.0759,  0.1657,
          0.1361, -0.0696,  0.1651, -0.1858, -0.1468,  0.1681,  0.0479,  0.1335,
          0.0235,  0.2120, -0.0235,  0.1609],
        [ 0.0635,  0.1810,  0.0416,  0.0279,  0.0858, -0.1147, -0.1616, -0.0342,
         -0.0314,  0.0787, -0.1648,  0.1374, -0.2007,  0.1755,  0.1366,  0.2135,
         -0.0906,  0.0360, -0.0893,  0.1323],
        [ 0.0214, -0.1772,  0.0940,  0.0115,  0.1899,  0.1250, -0.0032, -0.2001,
         -0.1171, -0.1114,  0.0976,  0.2077, -0.0590,  0.0543,  0.2160, -0.0612,
      


 50%|█████     | 1/2 [00:59<00:59, 59.61s/it]

tensor([-0.2157, -0.1423,  0.1574, -0.0399,  0.0700,  0.1900,  0.0724, -0.1432,
         0.2224, -0.1936])
[-0.21574047 -0.14234108  0.15735814 -0.03993347  0.0700005   0.1900393
  0.07242051 -0.14315319  0.22235888 -0.19361088]



100%|██████████| 2/2 [01:03<00:00, 31.74s/it]


(['0.weight', '0.bias'],
 [([<phe.paillier.EncryptedNumber at 0x1a5187df4c0>,
   (10, 20)),
  ([<phe.paillier.EncryptedNumber at 0x1a5188306d0>,
   (10,))])

In [36]:
decypt_order_dict(pair)

100%|██████████| 10/10 [00:00<00:00, 146.43it/s]


{'0.weight': array([[ 0.11518455, -0.14939436,  0.07464749,  0.13816199,  0.09538141,
         -0.21291763, -0.03641471, -0.03026898, -0.01343769,  0.09034711,
         -0.15266111,  0.15466586,  0.04393148, -0.17538685, -0.00593001,
         -0.16249502,  0.18362737, -0.13889664, -0.16882753,  0.00782479],
        [-0.19697411,  0.2144047 ,  0.12703305,  0.18246636,  0.02655205,
          0.15646845, -0.21153474, -0.21896769,  0.16667801,  0.19625062,
          0.21943474, -0.04816642, -0.1382429 ,  0.17164084,  0.06841075,
         -0.12224845, -0.07718995,  0.05788633, -0.17004006, -0.10357421],
        [ 0.18003663, -0.14414659, -0.10913046,  0.05112603,  0.04514816,
          0.04389238, -0.07587829,  0.16568843,  0.13612193, -0.06956978,
          0.16511205, -0.1858343 , -0.14678171,  0.16807595,  0.04793549,
          0.13348785,  0.02347019,  0.21201462, -0.02345286,  0.16089579],
        [ 0.06352219,  0.18099058,  0.0415678 ,  0.02794638,  0.08581647,
         -0.11467796, -

In [32]:
array = np.random.normal(0, 1, size=(10, 10))
encrypted = encrypt_random_vector(array)
decrypt_random_vector(encrypted)
array.dtype

KeyboardInterrupt: 

In [33]:
array = np.random.normal(0, 1, size=(10, 10))
array.dtype

dtype('float64')